In [45]:
# !pip install -U sentence-transformers

In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import evaluation
from sentence_transformers.cross_encoder import CrossEncoder
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import torch
import torch.nn as nn
import nltk
import re
nltk.download('words')
words = set(nltk.corpus.words.words())

def stemSentence_porter(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 


def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 


def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z0-9,.’]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    
    return text

def get_cosine_sim(model, df):
    scores = []
    for row in zip(df['tags'], df['reviews']):
        scores.append(util.cos_sim(model.encode(row[0]), model.encode(row[1])))
    return scores

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\czhao\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
movies = pd.read_csv('../data/tmdb_5000_data/tmdb_5000_movies.csv')
credits = pd.read_csv('../data/tmdb_5000_data/tmdb_5000_credits.csv') 
movies = movies.merge(credits,on='title')

reviews = pd.read_csv('../data/crawled_data/2022-11-19_movie_info_with_reviews.csv')
reviews = reviews[['id','reviews']]
reviews['reviews'] = reviews['reviews'].apply(lambda x: list(map(clean_text, x.split("\',"))))
movies = movies.merge(reviews,on='id', how='left')

movies.dropna(inplace=True)
movies['release_year'] = movies.release_date.apply(lambda x: x.split("-")[0]).astype(int)
movies = movies[movies['release_year']>1970]

movies = movies[['movie_id','title','release_year','overview','genres','keywords','cast','crew','reviews']]
movies.dropna(inplace=True)

movies['genres'] = movies['genres'].apply(convert)

movies['keywords'] = movies['keywords'].apply(convert)

movies['cast'] = movies['cast'].apply(convert)

movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

movies['crew'] = movies['crew'].apply(fetch_director)


movies['crew'] = movies['crew'].apply(lambda x: [" ".join(\
        w for w in nltk.wordpunct_tokenize(i) \
         if w.lower() in words or not w.isalpha()) for i in x])

# movies['cast'] = movies['cast'].apply(collapse)
# movies['crew'] = movies['crew'].apply(collapse)
# movies['genres'] = movies['genres'].apply(collapse)
# movies['keywords'] = movies['keywords'].apply(collapse)


# new['tags'] = new['tags'].apply(stemSentence_porter)


In [3]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['release_year'] = movies['release_year'].astype(str).apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['release_year'] + movies['keywords'] + movies['cast'] + movies['crew'] 
new = movies.drop(columns=['overview','genres','keywords','cast','crew','release_year'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new = new[(new['tags'].notnull())].reset_index(drop=True)
new = new[new["reviews"].str.len() != 0].reset_index(drop=True)

In [49]:
new = new.explode(['reviews'])
new = new[(new['reviews']!='') & (new['reviews'].str.len() >= 3)].reset_index(drop=True)
# new.to_csv("../tmdb_5000_data/Cleaned_Filtered_Plots_Reviews")

In [50]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")

def get_sentiment(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

C:\Users\czhao\anaconda3\lib\site-packages\transformers\models\auto\modeling_auto.py:1112: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
new['review_sentiment'] = new['reviews'].apply(lambda x: get_sentiment(x)[6:])

In [ ]:
new['sentiment_score'] = new['review_sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
new.sentiment_score.unique()

array([1., 0.])

In [ ]:
new = new[(new['reviews']!='') & (new['reviews'].str.len() >= 3)].reset_index(drop=True)

In [ ]:
new.to_csv("../data/tmdb_5000_data/tmdb_sentiment_reviews.csv")

In [2]:
new = pd.read_csv('../data/tmdb_5000_data/tmdb_sentiment_reviews.csv') 

In [3]:
from sentence_transformers import models, losses, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import evaluation

In [4]:
# microsoft/mpnet-base
# sentence-transformers/stsb-bert-large
word_embedding_model = models.Transformer('microsoft/mpnet-base').to(device)

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetModel: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [5]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 
                                  pooling_mode_mean_tokens=True, 
                                  pooling_mode_cls_token=False, 
                                  pooling_mode_max_tokens=False
                              )

In [6]:
# update the model
sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model]).to(device)

In [7]:
train_examples = []

for index, row in enumerate(zip(new['tags'], new['reviews'], new['sentiment_score'])):
#     train_examples.append(InputExample(texts=[row[0], row[1]], label = [scores[index]]))
    train_examples.append(InputExample(texts=[row[0], row[1]], label = row[2]))    

In [8]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=4)
# ContrastiveLoss
# MarginMSELoss
# OnlineContrastiveLoss
# TripletLoss
train_loss = losses.OnlineContrastiveLoss(sent_model)

In [9]:
# dev_mse = evaluation.EmbeddingSimilarityEvaluator(new['tags'].values, new['reviews'].values,\
#                                                   scores=scores)
dev_mse = evaluation.BinaryClassificationEvaluator(new['tags'].values, \
                          new['reviews'].values, \
                          new['sentiment_score'].values)

In [10]:
# dev_mse = evaluation.MSEEvaluator(new['tags'].values, new['reviews'].values,teacher_model=sent_model)

In [11]:
sent_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, output_path='result')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2725 [00:00<?, ?it/s]

In [15]:
descriptions = new['tags'].unique().tolist()
des_embeddings = []
for i,des in enumerate(descriptions):
    des_embeddings.append(sent_model.encode(des))

In [16]:
def recommend(query, model):
    #Compute cosine-similarities with all embeddings 
    model.eval()
    query_embedd = model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)
    top5_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][0:5]
    return top5_matches

In [19]:
query_show_des = 'dinosaurs'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(new.iloc[index,:])

movie_id                                                          503
title                                                        Poseidon
reviews             The movie is just what youd expect skimpy, alm...
tags                A packed cruise ship traveling the Atlantic is...
review_sentiment                                             negative
sentiment_score                                                     0
Name: 592, dtype: object
movie_id                                                        58595
title                                     Snow White and the Huntsman
reviews             Snow White and the Huntsman takes the ageless ...
tags                After the Evil Queen marries the King, she per...
review_sentiment                                             positive
sentiment_score                                                     1
Name: 466, dtype: object
movie_id                                                        50321
title                                   

In [18]:
query_show_des = 'purchases a large house that has a zoo'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(new.iloc[index,:])

movie_id                                                       258489
title                                            The Legend of Tarzan
reviews             The film’s faults center more around its inabi...
tags                Tarzan, having acclimated to life in London, i...
review_sentiment                                             negative
sentiment_score                                                     0
Name: 342, dtype: object
movie_id                                                        10195
title                                                            Thor
reviews             Sufficient enough variety to not fatigue. Full...
tags                Against his father Odin's will, The Mighty Tho...
review_sentiment                                             positive
sentiment_score                                                     1
Name: 730, dtype: object
movie_id                                                         6479
title                                   

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 97% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  2% |  5% |


In [ ]:
device

device(type='cuda')